##### Rôle des Décorateurs

#### 1. **`check_error`** :
   - **Rôle** : Intercepte et gère les exceptions qui se produisent lors de l'exécution de la fonction décorée, en affichant un message d'erreur avec le nom de la fonction et le message de l'exception.

#### 2. **`timing`** :
   - **Rôle** : Mesure le temps d'exécution de la fonction décorée et affiche la durée de l'exécution en secondes.

#### 3. **`connect_db`** :
   - **Rôle** : Gère la connexion à une base de données SQLite. Il établit la connexion, crée un curseur, passe la connexion et le curseur à la fonction décorée, puis s'assure que la connexion est validée et fermée après l'exécution de la fonction.


In [1]:
#declarations des decorateurs selon les besoins:
def check_error(function):
    def wrapper(*args, **kwargs):
        try:
            return function(*args, **kwargs)  
        except Exception as e: 
            print("Fonction d'erreur: {}, message: {}".format(function.__name__,str(e)))  
    return wrapper

def timing(function):
    def wrapper(*args, **kwargs):
        start:float = time.time()
        result = function(*args, **kwargs)
        end:float = time.time()
        print("Temps d'exécution de {}: {:.4f} secondes".format(function.__name__ , end - start))
        return result
    return wrapper

def connect_db(function):
    def wrapper(*args , **kwargs) :
        try :
            connection:sqlite3.connect = sqlite3.connect(kwargs["table"])
            cursor:connection.cursor = connection.cursor()
            kwargs['connection'] = connection
            kwargs['cursor'] = cursor
            print("connection avec succes") 
            return function(*args , **kwargs)
        except Exception as e :
            print("Fonction d'erreur: {}, message: {}".format(function.__name__,str(e)))
        
        finally :
            connection.commit()
            connection.close()
    return wrapper

In [8]:
import pandas as pd
import h5py as hdf5
import os
import time
import sqlite3
from typing import List, Dict

#1. Échantillonnage et Sous-ensemble de Données:
@check_error  
def load_csv_file_chunks(path_file:str, use_cols:List[str], dtypes:Dict[str,str], chunk_size:int,fraction:float=0.01)->pd.DataFrame:
    data:pd.DataFrame = pd.DataFrame()
    for ch in pd.read_csv(path_file , usecols= use_cols, dtype= dtypes ,chunksize=chunk_size):
        data = pd.concat([data, ch], ignore_index=True)
    print("donnés chargé savec succe!")
    return data.sample(frac=fraction, random_state = 2)

data_sales:pd.DataFrame = pd.DataFrame()
cols:List[str] = ["customer_id","product_id","quantity","price"]
dtypes:Dict[str,str] = {
    'customer_id' : 'uint32',  
    'product_id' : 'uint16',        
    'quantity': 'uint8',          
    'price': 'float32',                  
}
chunk_size:int = 100000
data_sales = load_csv_file_chunks('sales_data.csv',cols,dtypes,chunk_size)
print(data_sales.info())


donnés chargé savec succe!
<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 309190 to 194758
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  10000 non-null  uint32 
 1   product_id   10000 non-null  uint16 
 2   quantity     10000 non-null  uint8  
 3   price        10000 non-null  float32
dtypes: float32(1), uint16(1), uint32(1), uint8(1)
memory usage: 185.5 KB
None


In [7]:
#2. Conversion en Formats de Fichiers Efficaces:
@check_error
def convert_to_feather_or_parquet(data:pd.DataFrame,type_convert_feather:bool=True)->None:  
    if type_convert_feather == True:
        print("Donnés converté au feather")
        data.to_feather('data.feather')
    if type_convert_feather == False:
        print("Donnés converté au parquet") 
        data.to_parquet('data.parquet')

@check_error
@timing
def read_files_feather_parquet_csv(file_name:str)->any:
    choice:List[str] = file_name.split(".")  

    match choice[-1]:
        case "feather":
            print("Chargement d'un fichier ",choice[-1])
            return pd.read_feather(file_name)
        case "parquet":
            print("Chargement d'un fichier ",choice[-1])
            return pd.read_parquet(file_name)
        case "csv":
            print("Chargement d'un fichier ",choice[-1])
            return pd.read_csv(file_name)
        case _:
            print("Ce type n'est pas un type (csv,feather ou paquet).")
            return None
#conveersion csv:
data_sales.to_csv('data.csv', index=False)
#conversion feather:
convert_to_feather_or_parquet(data_sales,type_convert_feather=True)
#conversion parquet:
convert_to_feather_or_parquet(data_sales,type_convert_feather=False)

#comparaison des tailles ( parquet, feateher et csv):
# Obtenir la taille des fichiers en octets
file_size_csv = os.path.getsize('data.csv')
file_size_feather = os.path.getsize('data.feather')
file_size_parquet = os.path.getsize('data.parquet')

print("Taille du fichier Parquet: {:.2f} octets".format(file_size_parquet))
print("Taille du fichier CSV: {:.2f} octets".format(file_size_csv))
print("Taille du fichier Feather: {:.2f} octets".format(file_size_feather))


#mesure du temps de chargement des fichiers:
print("#"*40)
csv_data:pd.DataFrame = read_files_feather_parquet_csv("data.csv")
print("#"*40)
feather_data:pd.DataFrame = read_files_feather_parquet_csv("data.feather")
print("#"*40)
parquet_data:pd.DataFrame = read_files_feather_parquet_csv("data.parquet")
csv_data.shape


Donnés converté au feather
Donnés converté au parquet
Taille du fichier Parquet: 226743.00 octets
Taille du fichier CSV: 205988.00 octets
Taille du fichier Feather: 158714.00 octets
########################################
Chargement d'un fichier  csv
Temps d'exécution de read_files_feather_parquet_csv: 0.0387 secondes
########################################
Chargement d'un fichier  feather
Temps d'exécution de read_files_feather_parquet_csv: 0.0145 secondes
########################################
Chargement d'un fichier  parquet
Temps d'exécution de read_files_feather_parquet_csv: 0.0161 secondes


(10000, 4)

In [267]:
import h5py
#Utilisation de HDF5:
@check_error
def file_hdf5( file_name:str , data:pd.DataFrame , table_name:str)->None:
        with hdf5.File(file_name, "a") as hd5_file:
            hd5_file.create_dataset(table_name, data=data.values)
#stockez l'échantillon de données dans une table appelée sales_sample.

file_hdf5("sales_data.h5", data_sales, 'sales_sample')
#Ajoutez une deuxième table contenant les transactions dont le prix est supérieur à 100 DH.
data_transaction_supp_100 = data_sales[data_sales['price'] > 100]
file_hdf5("sales_data.h5", data_transaction_supp_100, 'sales_supp_100')
#Lire les données du table 'sales_sample' et affich 5 premier :
with h5py.File('sales_data.h5', 'r') as hdf:
    print(hdf['sales_sample'][0:5])

[[1.82370000e+04 5.87200000e+03 5.00000000e+00 2.71320007e+02]
 [2.16230000e+04 4.43700000e+03 2.00000000e+00 2.84260010e+02]
 [4.53030000e+04 6.76700000e+03 1.00000000e+01 3.16850006e+02]
 [2.34910000e+04 5.13400000e+03 9.00000000e+00 8.67500000e+01]
 [1.68730000e+04 6.68600000e+03 1.00000000e+00 1.84309998e+02]]


In [311]:
#4. Lecture par Morceaux:
@check_error
def read_csv_file_by_parts(file_name:str,parts:int):
    for part in pd.read_csv(file_name, chunksize=parts):
        yield part
#Lisez le fichier sales_data.csv par morceaux de 100 000 lignes et filtrez les transactions ayant une quantité supérieure à 10.
#et combinaison des dataframe filtrer.
data_transaction:pd.DataFrame = pd.DataFrame()
for part in read_csv_file_by_parts("sales_data.csv",100000):
    data_transaction = pd.concat([data_transaction , part[part['quantity'] > 10]] , ignore_index=True)
print("donnees filtrer :", len(data_transaction))
#calculez le total des ventes (quantité * prix) pour ces transactions.
data_transaction['total_value'] = data_transaction['price'] * data_transaction['quantity']
total = data_transaction['total_value'].sum()
print("total des ventes: ",total)

donnees filtrer : 0
total des ventes:  0.0


In [373]:
#5. Chargement dans une Base de Données:
@check_error
@connect_db
def copy_csv_to_sql_db(connection:sqlite3.connect , data:pd.DataFrame, table:str ,cursor = None)->None:
    data.to_sql(table, connection , if_exists='replace' , index=False )
    print("donnes csv to sql est bien fait!")


@check_error
@connect_db
def select_transaction_group_by_eur(cursor , connection:sqlite3.connect = None , table:str=None) :
    stm = cursor.execute(""" SELECT * FROM sales WHERE region='Europe' and price > 50   """)
    data = stm.fetchall()
    return  data

#Créez une base de données SQLite (sales.db) et chargez l'intégralité du fichier
#sales_data.csv dans une table appelée sales.
copy_csv_to_sql_db(table="sales",data=csv_data)
#Exécutez une requête SQL pour extraire les transactions ayant eu lieu dans la région
#"Europe" et dont le prix est supérieur à 50 DH.
data_region_price:pd.DataFrame = pd.DataFrame(select_transaction_group_by_eur(table="sales"), columns=csv_data.columns)
print('donnees region \'Europe\' et price supp a 50 :')
print(data_region_price)
#Calculez le total des ventes pour ces transactions et affichez le résultat.
data_region_price['total_value'] = data_region_price['price'] * data_region_price['quantity']
total = data_region_price['total_value'].sum()
print("total transaction: ",total)


connection avec succes
donnes csv to sql est bien fait!
connection avec succes
donnees region 'Europe' et price supp a 50 :
        transaction_id  customer_id  product_id  quantity   price  \
0                    3        76821        3079         5  473.96   
1                    4        54887        9037         4  466.34   
2                    9        44132        1056         5  405.97   
3                   14        64821        3710         3  438.17   
4                   16        59736        4594         2  433.96   
...                ...          ...         ...       ...     ...   
153039          999950        38660        6416         7   78.04   
153040          999959         7434        9012         4  474.91   
153041          999962        38987        5372         1  474.39   
153042          999989        56292        4593         4  352.91   
153043          999998        54332        3891        10  186.24   

       transaction_date  region  
0            

connection avec succes
             0      1     2   3       4           5       6
0            3  76821  3079   5  473.96  2023-09-20  Europe
1            4  54887  9037   4  466.34  2020-12-11  Europe
2            9  44132  1056   5  405.97  2020-06-16  Europe
3           14  64821  3710   3  438.17  2021-06-04  Europe
4           16  59736  4594   2  433.96  2023-11-21  Europe
...        ...    ...   ...  ..     ...         ...     ...
153039  999950  38660  6416   7   78.04  2023-07-04  Europe
153040  999959   7434  9012   4  474.91  2022-02-09  Europe
153041  999962  38987  5372   1  474.39  2020-12-21  Europe
153042  999989  56292  4593   4  352.91  2023-09-23  Europe
153043  999998  54332  3891  10  186.24  2023-03-25  Europe

[153044 rows x 7 columns]


In [194]:
import h5py
import pandas as pd
import numpy as np


data = {
    'TransactionID': [1, 2, 3, 4, 5],
    'Prix': [50, 150, 80, 120, 200],
    'Quantité': [1, 2, 1, 3, 1]
}
normal_data = pd.DataFrame(data)

# Convertir les types avec numpy
normal_data['TransactionID'] = normal_data['TransactionID'].astype(np.int32) 
normal_data['Prix'] = normal_data['Prix'].astype(np.float64)
normal_data['Quantité'] = normal_data['Quantité'].astype(np.int32)

# Filtrer les transactions dont le prix est supérieur à 100
data_transaction_supp_100 = normal_data[normal_data['Prix'] > 100]

# Créer un fichier HDF5 et y stocker les deux tables
with h5py.File("sales_data.h5", "w") as hd5_file:
    hd5_file.create_dataset('sales_sample', data=normal_data)
    hd5_file.create_dataset('sales_high_transaction', data=data_transaction_supp_100)

with h5py.File('sales_data.h5', 'r') as hdf:
    # Lire les données du table 'sales_sample'
    print(hdf['sales_sample'][0:5])


[[  1.  50.   1.]
 [  2. 150.   2.]
 [  3.  80.   1.]
 [  4. 120.   3.]
 [  5. 200.   1.]]


In [36]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
# Paramètres pour la génération de données
num_rows = 1_000_000 # 1 million de lignes
regions = ['North America', 'Europe', 'Asia', 'South America', 'Africa', 'Australia']
start_date = datetime(2020, 1, 1) # Date de début pour les transactions
end_date = datetime(2023, 12, 31) # Date de fin pour les transactions
# Génération des données
np.random.seed(42) # Pour la reproductibilité
# transaction_id : Identifiant unique
transaction_ids = np.arange(1, num_rows + 1)
# customer_id : Identifiant client aléatoire entre 1 et 100 000
customer_ids = np.random.randint(1, 100_001, num_rows)
# product_id : Identifiant produit aléatoire entre 1 et 10 000
product_ids = np.random.randint(1, 10_001, num_rows)
# quantity : Quantité aléatoire entre 1 et 10
quantities = np.random.randint(1, 11, num_rows)
# price : Prix aléatoire entre 10 et 500 (avec 2 décimales)
prices = np.round(np.random.uniform(10, 500, num_rows), 2)
# transaction_date : Date aléatoire entre start_date et end_date
date_range = (end_date - start_date).days
transaction_dates = [start_date + timedelta(days=np.random.randint(0, date_range)) for _ in range(num_rows)]
# region : Région aléatoire parmi la liste définie
regions_data = np.random.choice(regions, num_rows)
# Création du DataFrame
df = pd.DataFrame({
'transaction_id': transaction_ids,
'customer_id': customer_ids,
'product_id': product_ids,
'quantity': quantities,
'price': prices,
'transaction_date': transaction_dates,
'region': regions_data
})
# Affichage des 5 premières lignes
print(df.head())
# Sauvegarde du DataFrame dans un fichier CSV
df.to_csv('sales_data.csv', index=False)
print("Fichier 'sales_data.csv' sauvegardé avec succès.")

   transaction_id  customer_id  product_id  quantity   price transaction_date  \
0               1        15796         111         7  336.48       2023-10-03   
1               2          861        3214         3  471.86       2022-06-01   
2               3        76821        3079         5  473.96       2023-09-20   
3               4        54887        9037         4  466.34       2020-12-11   
4               5         6266        7572         7   39.62       2023-07-04   

          region  
0  North America  
1      Australia  
2         Europe  
3         Europe  
4  North America  
Fichier 'sales_data.csv' sauvegardé avec succès.


In [ ]:
transaction_id,